<a href="https://colab.research.google.com/github/sumanyurosha/tensorflow-specialization/blob/master/Hands-on%20ML/chapter12/Customizing_Models_and_Training_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import tensorflow as tf
from tensorflow import keras

# **Custom Loss Functions**

In [3]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In [4]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_error_small = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_error_small, squared_loss, linear_loss)

In [5]:
model.compile(loss=huber_fn, optimizer='nadam')

In [8]:
tf.keras.models.save_model(model, 'my_model_with_custom_loss.h5')

In [10]:
model = tf.keras.models.load_model('my_model_with_custom_loss.h5', 
                            custom_objects={'huber_fn': huber_fn})

# What if you want to add a threshold to measure whether the error is small or not? 

In [12]:
def create_hubor(threshold=1):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_error_small = tf.abs(error) < 1
        squared_loss = tf.square(error) / 2
        linear_loss = threshold * tf.abs(error) - threshold**2 / 2
        return tf.where(is_error_small, squared_loss, linear_loss)
    return huber_fn


In [15]:
model.compile(loss=create_hubor(2.0), optimizer='nadam')

In [16]:
tf.keras.models.save_model(model, 'my_model_with_custom_loss_2.h5')

In [17]:
model = tf.keras.models.load_model('my_model_with_custom_loss_2.h5', 
                                   custom_objects={'huber_fn': create_hubor(2.0)})

# You have to pass the threshold whenever you load the model, so we can overcome this problem by subclassing keras.losses.Loss class

In [28]:
class HuborLoss(keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)

    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_error_small = tf.abs(error) < 1
        squared_loss = tf.square(error) / 2
        linear_loss = threshold * tf.abs(error) - threshold**2 / 2
        return tf.where(is_error_small, squared_loss, linear_loss)
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 'threshold': self.threshold}

In [29]:
model.compile(loss=HuborLoss(2.), optimizer='nadam')

In [30]:
keras.models.save_model(model, 'my_model_with_custom_loss_3.h5')

In [31]:
model = keras.models.load_model('my_model_with_custom_loss_3.h5', 
                                custom_objects={'HuborLoss': HuborLoss})

# **Custom Activation**

In [32]:
def my_softplus(z):
    return tf.log(tf.exp(z) + 1.)

# **Custom Initializer**

In [34]:
def my_glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / shape[0] + shape[1])
    return tf.random.normal(shape, stdev=stdev, dtype=dtype)

# Custom Regularizer

In [35]:
def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

# **Custom Constraint**

In [37]:
def my_postivie_weight(weights):
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

In [38]:
layer = keras.layers.Dense(30, 
                           activation=my_softplus,
                           kernel_initializer=my_glorot_initializer,
                           kernel_regularizer=my_l1_regularizer,
                           kernel_constraint=my_postivie_weight)

**If for a function we need to save the hyperparameter with the model, we will have to subclass it**

In [39]:
class MyL1Regularizer(keras.regularizers.Regularizer):
    def __init__(self, factor):
        super().__init__()
        self.factor = factor

    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))

    def get_config(self):
        return {'factor': self.factor}

# **Custom Metrics**


In [41]:
class HuborMetric(keras.metrics.Metric):
    def __init__(self, threshold=1., **kwargs):
        super().__init(**kwargs)
        self.threshold = threshold
        self.hubor_fn = create_hubor(threshold)
        self.total = self.add_weight('total', initializer='zeros'),
        self.count = self.add_weight('count', initializer='zeros')
    
    def update_state(self, y_true, y_pred):
        metric = self.hubor_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.total.assign_add(tf.cast(tf.size(y_true)), dtype=tf.float32)

    def result(self):
        return self.total / self.count

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 'threshold': self.threshold}
